In [22]:
import pandas as pd
import re

def base_zattini():
    arquivo_zattini = input('Digite o caminho do seu arquivo da zattini: ').strip('\'"')
    zattini_excel = pd.read_excel(arquivo_zattini)
    zattini_df = pd.DataFrame(zattini_excel)
    zattini_df = zattini_df.rename(columns={'Sku Seller': 'Código'})
    
    # Função para extrair números de uma string
    def extract_numbers(text):
        match = re.search(r'\d+', text)
        if match:
            return int(match.group())
        return None
    
    # Aplicar a função personalizada à coluna 'Código'
    zattini_df['Código'] = zattini_df['Código'].apply(extract_numbers)
    
    return zattini_df

# Chamar a função
zattini = base_zattini()

# Verificar o resultado
zattini


c:\Users\anton\AppData\Local\Programs\Python\Python312\Lib\site-packages\openpyxl\styles\stylesheet.py:226: UserWarning: Workbook contains no default style, apply openpyxl's default
  warn("Workbook contains no default style, apply openpyxl's default")


,Código,Preço De,Preço Por
0,246258,239.99,159.99
1,246267,169.99,129.99
2,246255,239.99,159.99
3,246256,239.99,159.99
4,246265,169.99,129.99
...,...,...,...
31798,218806,399.99,389.99
31799,222191,309.99,307.00
31800,248710,237.00,229.99
31801,248709,237.00,229.99


In [14]:
def base_bagy():
    """
    Processa um arquivo JSON da Bagy e retorna um DataFrame pandas formatado.

    Este método solicita ao usuário que insira o caminho para um arquivo JSON exportado da plataforma Bagy. 
    Em seguida, ele lê o arquivo JSON, seleciona e renomeia colunas específicas, reorganiza as colunas e 
    remove quaisquer linhas que não possuam um código de variação ('Código'). Finalmente, converte a coluna 
    'Código' para o tipo inteiro e retorna o DataFrame resultante.

    Retorna:
        pandas.DataFrame: DataFrame contendo os dados processados do arquivo JSON da Bagy com as colunas:
            - 'SKU Pai': ID externo do produto
            - 'Código': SKU da variação do produto, convertido para inteiro
            - 'Marca': Nome da marca do produto
            - 'Nome': Nome do produto
            - 'Estoque': Quantidade em estoque
            - 'Preço De': Preço original do produto (antes de desconto)
            - 'Preço Por': Preço atual do produto (após desconto)
    """
    arquivo_json = input('Digite o caminho do seu arquivo da Bagy: ').strip('\'"')
    bagy_json = pd.read_json(arquivo_json)
    bagy_json = bagy_json[['Brands → Name', 'Variations → Sku', 'Price', 'Price Compare', 'Name', 'Stocks → Balance', 'External ID']]
    bagy_json = bagy_json.rename(columns={'Brands → Name': 'Marca', 'Variations → Sku': 'Código', 'Price Compare': 'Preço De', 'Price': 'Preço Por', 'Name': 'Nome', 'Stocks → Balance': 'Estoque', 'External ID': 'SKU Pai'})
    bagy_json = bagy_json.reindex(columns=["SKU Pai", "Código", "Marca", "Nome", "Estoque", "Preço De", "Preço Por"])
    bagy_json = bagy_json.dropna(subset=['Código'])
    bagy_json['Código'] = bagy_json['Código'].astype(int)
    return bagy_json

bagy = base_bagy()
bagy

,SKU Pai,Código,Marca,Nome,Estoque,Preço De,Preço Por
0,196988,196989,Aeropostale,Bermuda Aeropostale D Água Masculina Logo Azul...,0,279.99,159.99
1,196988,196990,Aeropostale,Bermuda Aeropostale D Água Masculina Logo Azul...,0,279.99,159.99
2,196988,196991,Aeropostale,Bermuda Aeropostale D Água Masculina Logo Azul...,0,279.99,159.99
3,196988,196992,Aeropostale,Bermuda Aeropostale D Água Masculina Logo Azul...,0,279.99,159.99
4,253696,253697,Aeropostale,Bermuda Aeropostale D'Água Masculina Bold Logo...,3,299.99,229.99
...,...,...,...,...,...,...,...
81982,206080,206081,John John,Tricot John John Masculino Travis Red JJ Preta,0,398.00,299.99
81983,206080,206082,John John,Tricot John John Masculino Travis Red JJ Preta,1,398.00,299.99
81984,206080,206083,John John,Tricot John John Masculino Travis Red JJ Preta,0,398.00,299.99
81985,206080,206084,John John,Tricot John John Masculino Travis Red JJ Preta,0,398.00,299.99


In [31]:
def exportar_zattini(bagy, marketplace):
     """
    Exporta dados de produtos para a Zattini, filtrando marcas proibidas e ajustando status de cadastro.

    Esta função realiza os seguintes passos:
    1. Remove produtos de marcas proibidas específicas.
    2. Define o status de cadastro para 'Sem Cadastro'.
    3. Atualiza o status de cadastro para 'Cadastrado' para produtos cujos códigos estão presentes no dataframe do marketplace.
    4. Mescla dados adicionais do marketplace no dataframe principal com base no código do produto.
    5. Remove as colunas 'Código' e 'Marca'.
    6. Renomeia as colunas de preços para distinguir entre preços do Bagy e da Zattini.
    7. Agrega os dados por 'SKU Pai', somando os estoques e mantendo o primeiro valor encontrado para as demais colunas.
    8. Ordena os dados pelo estoque em ordem decrescente.
    9. Remove linhas específicas com os índices 4604 e 3555, que correspondem a sacola plástica e embalagem de presente.
    
    Parâmetros:
    bagy (pd.DataFrame): DataFrame contendo os dados dos produtos do Bagy.
    marketplace (pd.DataFrame): DataFrame contendo os dados dos produtos do marketplace da Zattini.

    Retorna:
    pd.DataFrame: DataFrame processado e pronto para exportação para a Zattini, com as marcas proibidas removidas, status de cadastro atualizado e dados agregados por 'SKU Pai'.
    """
    marcas_proibidas_zattini = ['Aeropostale', 'Lacoste', 'Levis', 'Nike', 'Osklen', 'Polo Ralph Lauren', 'Reserva']
    df_exportar_zattini = bagy[~bagy['Marca'].isin(marcas_proibidas_zattini)]
    df_exportar_zattini.loc[:, 'Status Cadastro'] = 'Sem Cadastro'
    df_exportar_zattini.loc[df_exportar_zattini['Código'].isin(marketplace['Código']), 'Status Cadastro'] = 'Cadastrado'
    df_exportar_zattini = df_exportar_zattini.merge(marketplace, on='Código', how='left')
    df_exportar_zattini = df_exportar_zattini.drop(columns=['Código', 'Marca'])
    df_exportar_zattini = df_exportar_zattini.rename(columns= {'Preço De_x': 'Preço De Bagy', 'Preço Por_x': 'Preço Por Bagy', 'Preço De_y': 'Preço De Ztn', 'Preço Por_y': 'Preço Por Ztn' })
    df_exportar_zattini = df_exportar_zattini.groupby('SKU Pai').agg({
        'Nome': 'first',
        'Estoque': 'sum',
        'Preço De Bagy': 'first',
        'Preço Por Bagy': 'first',
        'Status Cadastro': 'first',
        'Preço De Ztn': 'first',
        'Preço Por Ztn': 'first'
    }).reset_index()
    df_exportar_zattini = df_exportar_zattini.sort_values(by='Estoque', ascending=False)
    df_exportar_zattini.drop([4604, 3555], inplace=True) #removendo sacola plastica e embalagem de presente
    return df_exportar_zattini


exportar_zattini(bagy, zattini)

C:\Users\anton\AppData\Local\Temp\ipykernel_24316\2438519149.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_exportar_zattini.loc[:, 'Status Cadastro'] = 'Sem Cadastro'


,SKU Pai,Nome,Estoque,Preço De Bagy,Preço Por Bagy,Status Cadastro,Preço De Ztn,Preço Por Ztn
4137,231492,Camiseta Tommy Hilfiger Masculina Essential Co...,2153,199.99,136.83,Cadastrado,299.99,199.99
12177,299830,Camisa Aramis Masculina Cosmo Stretch Algodão ...,1185,499.99,210.52,Cadastrado,499.99,249.99
12178,299835,Camisa Aramis Masculina Cosmo Stretch Algodão ...,1161,499.99,210.52,Cadastrado,499.99,249.99
12179,299840,Camisa Aramis Masculina Cosmo Stretch Algodão ...,855,499.99,210.52,Sem Cadastro,NaN,NaN
12180,299845,Camisa Aramis Masculina Cosmo Stretch Algodão ...,849,499.99,210.52,Cadastrado,499.99,249.99
...,...,...,...,...,...,...,...,...
3958,230000,Polo Aramis Masculina Malha Poá Bordô,0,269.99,139.99,Sem Cadastro,NaN,NaN
5256,240290,Cuecas Calvin Klein Brief Cotton Print Preta P...,-1,199.99,169.99,Cadastrado,169.99,169.99
4417,233524,Polo Tommy Jeans Masculina Regular Badge Light...,-1,329.99,299.99,Cadastrado,349.99,299.99
5468,241722,Boné New Era 9Forty MLB Yankees Veranito BLK P...,-1,249.99,199.99,Sem Cadastro,NaN,NaN
